In [1]:
#Import packages
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries Imported')

Libraries Imported


In [2]:
df = pd.read_csv(r'C:\Users\Nick\Desktop\zip_code_database.csv')

In [3]:
df.columns

Index(['zip', 'type', 'decommissioned', 'primary_city', 'acceptable_cities',
       'unacceptable_cities', 'state', 'county', 'timezone', 'area_codes',
       'world_region', 'country', 'latitude', 'longitude',
       'irs_estimated_population_2015'],
      dtype='object')

In [4]:
df.shape

(42632, 15)

In [5]:
df.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [6]:
df_state = df.loc[(df['state']=='FL')|(df['state']=='MO')|(df['state']=='GA')|(df['state']=='TX')]

In [7]:
df_cities = df_state.loc[(df_state['primary_city']=='Fort Lauderdale')|(df_state['primary_city']=='Kansas City')|(df_state['primary_city']=='Atlanta')|(df_state['primary_city']=='Dallas')]

In [8]:
df_cities['primary_city'].unique()

array(['Dallas', 'Atlanta', 'Fort Lauderdale', 'Kansas City'],
      dtype=object)

In [9]:
df_cities.shape

(356, 15)

In [10]:
df_cities.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
12964,30132,STANDARD,0,Dallas,NaN,NaN,GA,Paulding County,America/New_York,"678,770",NaN,US,33.91,-84.83,34040
12987,30157,STANDARD,0,Dallas,NaN,NaN,GA,Paulding County,America/New_York,"678,770",NaN,US,33.88,-84.87,42710
13079,30301,PO BOX,0,Atlanta,NaN,Atl,GA,Fulton County,America/New_York,404,NaN,US,33.85,-84.39,1009
13080,30302,PO BOX,0,Atlanta,NaN,Atl,GA,Fulton County,America/New_York,404,NaN,US,33.75,-84.38,774
13081,30303,STANDARD,0,Atlanta,NaN,"Atl, Georgia State University",GA,Fulton County,America/New_York,"470,404,678",NaN,US,33.75,-84.39,1820


In [11]:
df_cities.isna().sum()

zip                                0
type                               0
decommissioned                     0
primary_city                       0
acceptable_cities                258
unacceptable_cities              168
state                              0
county                             1
timezone                           0
area_codes                         1
world_region                     356
country                            0
latitude                           0
longitude                          0
irs_estimated_population_2015      0
dtype: int64

In [12]:
df_cities = df_cities.drop(['world_region','acceptable_cities','unacceptable_cities'], axis = 1)

In [13]:
df_cities.isna().sum()

zip                              0
type                             0
decommissioned                   0
primary_city                     0
state                            0
county                           1
timezone                         0
area_codes                       1
country                          0
latitude                         0
longitude                        0
irs_estimated_population_2015    0
dtype: int64

In [14]:
df_cities.columns

Index(['zip', 'type', 'decommissioned', 'primary_city', 'state', 'county',
       'timezone', 'area_codes', 'country', 'latitude', 'longitude',
       'irs_estimated_population_2015'],
      dtype='object')

In [15]:
df_cities.head()

,zip,type,decommissioned,primary_city,state,county,timezone,area_codes,country,latitude,longitude,irs_estimated_population_2015
12964,30132,STANDARD,0,Dallas,GA,Paulding County,America/New_York,"678,770",US,33.91,-84.83,34040
12987,30157,STANDARD,0,Dallas,GA,Paulding County,America/New_York,"678,770",US,33.88,-84.87,42710
13079,30301,PO BOX,0,Atlanta,GA,Fulton County,America/New_York,404,US,33.85,-84.39,1009
13080,30302,PO BOX,0,Atlanta,GA,Fulton County,America/New_York,404,US,33.75,-84.38,774
13081,30303,STANDARD,0,Atlanta,GA,Fulton County,America/New_York,"470,404,678",US,33.75,-84.39,1820


In [16]:
import folium
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [17]:
address = 'Florida'

geolocator = Nominatim(user_agent='FL_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Florida are {}, {}'.format(latitude, longitude))

The geographical coordinate of Florida are 27.7567667, -81.4639835


In [18]:
map_states = folium.Map(location=[latitude, longitude],zoom_start=10)

In [82]:
for lat, lng, primary_city,county in zip(df_cities['latitude'], df_cities['longitude'],df_cities['primary_city'], df_cities['county']):
    label = '{}, {}'.format(primary_city, county)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_states)
    
map_states
    

In [19]:
CLIENT_ID = 'COSXCEGQOUXYF5X4I1PSPFOPJFIUD1H4KWWD2ULYA2KWPBIK' # your Foursquare ID
CLIENT_SECRET = '4Q5JT1Z0O5I3FG0HFIB1KW1Q5SIMTM0UTHWIFGQTLRRM5V4U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: COSXCEGQOUXYF5X4I1PSPFOPJFIUD1H4KWWD2ULYA2KWPBIK
CLIENT_SECRET:4Q5JT1Z0O5I3FG0HFIB1KW1Q5SIMTM0UTHWIFGQTLRRM5V4U


In [20]:
df_cities = df_cities.reset_index()

In [21]:
county_latitude = df_cities.loc[0, 'latitude']
county_longitude = df_cities.loc[0, 'longitude']

county_name = df_cities.loc[0, 'county']

print('Latitude and Longitude values of {} are {}, {}.'.format(county_name, county_latitude, county_longitude))

Latitude and Longitude values of Paulding County are 33.91, -84.83.


In [25]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    county_latitude, 
    county_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=COSXCEGQOUXYF5X4I1PSPFOPJFIUD1H4KWWD2ULYA2KWPBIK&client_secret=4Q5JT1Z0O5I3FG0HFIB1KW1Q5SIMTM0UTHWIFGQTLRRM5V4U&v=20180605&ll=33.91,-84.83&radius=500&limit=100'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6035c24624672833e33c0577'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Dallas',
  'headerFullLocation': 'Dallas',
  'headerLocationGranularity': 'city',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 33.9145000045, 'lng': -84.82458787602502},
   'sw': {'lat': 33.905499995499994, 'lng': -84.83541212397498}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb74e94f562ef3b2e373197',
       'name': 'Silver Comet Trail - Paulding Chamber',
       'location': {'lat': 33.90723624044473,
        'lng': -84.82885180044025,
        'labeledLatLngs': [{'label': 'display',
          'lat': 33.90723624044473

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
            
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-28-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Silver Comet Trail - Paulding Chamber,Trail,33.907236,-84.828852


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['County', 
                  'County Latitude', 
                  'County Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
venues = getNearbyVenues(names= df_cities['county'],
                                   latitudes= df_cities['latitude'],
                                   longitudes=df_cities['longitude']
                                  )

Paulding County
Paulding County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
DeKalb County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
DeKalb County
DeKalb County
Fulton County
DeKalb County
Clayton County
Fulton County
DeKalb County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
DeKalb County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
DeKalb County
Cobb County
DeKalb County
DeKalb County
Fulton County
Fulton County
Fulton County
DeKalb County
DeKalb County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
DeKalb County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Fulton County
Ful

In [32]:
venues.shape

(4900, 7)

In [33]:
venues.head()

,County,County Latitude,County Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paulding County,33.91,-84.83,Silver Comet Trail - Paulding Chamber,33.907236,-84.828852,Trail
1,Fulton County,33.85,-84.39,Tres Belle,33.853870,-84.388878,Hotel
2,Fulton County,33.85,-84.39,Knollwood Nook,33.850999,-84.394995,Bar
3,Fulton County,33.75,-84.38,Nick's Food To Go,33.747492,-84.379749,Greek Restaurant
4,Fulton County,33.75,-84.38,Village Theatre,33.750042,-84.377155,Comedy Club


In [34]:
venues.groupby('County').count()

,County Latitude,County Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
County,,,,,,
Broward County,788,788,788,788,788,788
Cass County,5,5,5,5,5,5
Clay County,29,29,29,29,29,29
Clayton County,2,2,2,2,2,2
Cobb County,8,8,8,8,8,8
Collin County,15,15,15,15,15,15
Dallas County,1410,1410,1410,1410,1410,1410
DeKalb County,181,181,181,181,181,181
Denton County,10,10,10,10,10,10


In [36]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 303 unique categories.


In [37]:
venues.columns

Index(['County', 'County Latitude', 'County Longitude', 'Venue',
       'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [43]:
Venues_onehot = pd.get_dummies(venues[['Venue Category']],prefix="",prefix_sep="")
Venues_onehot['County'] = venues['County']

In [45]:
front = Venues_onehot['County']
Venues_onehot.drop(labels=['County'],axis=1,inplace=True)
Venues_onehot.insert(0,'County',front)

In [46]:
Venues_onehot.head()

,County,ATM,Accessories Store,Advertising Agency,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Paulding County,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Fulton County,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Fulton County,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fulton County,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Fulton County,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
Venues_onehot.shape

(4900, 304)

In [48]:
Venues_grouped = Venues_onehot.groupby('County').mean().reset_index()

In [49]:
Venues_grouped

,County,ATM,Accessories Store,Advertising Agency,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Broward County,0.000000,0.007614,0.001269,0.001269,0.001269,0.001269,0.011421,0.000000,0.000000,...,0.000000,0.000000,0.001269,0.001269,0.000000,0.002538,0.000000,0.001269,0.006345,0.001269
1,Cass County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Clay County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Clayton County,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Cobb County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Collin County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Dallas County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014894,0.000000,0.002837,...,0.001418,0.000000,0.004255,0.001418,0.001418,0.000000,0.000709,0.006383,0.000000,0.002128
7,DeKalb County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016575,0.005525,0.011050,...,0.005525,0.005525,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005525,0.005525
8,Denton County,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Fulton County,0.000000,0.002014,0.000000,0.002014,0.000000,0.000000,0.036254,0.000000,0.003021,...,0.004028,0.000000,0.001007,0.003021,0.001007,0.000000,0.001007,0.002014,0.007049,0.004028


In [50]:
Venues_grouped.shape

(14, 304)

In [57]:
num_top_venues = 5

for c in Venues_grouped['County']:
    print("----"+c+"----")
    temp = Venues_grouped[Venues_grouped['County']==c].T.reset_index()
    temp.columns = ['venues','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Broward County----
                venues  freq
0  Rental Car Location  0.05
1       Clothing Store  0.04
2       Ice Cream Shop  0.04
3   Athletics & Sports  0.03
4              Brewery  0.03


----Cass County----
             venues  freq
0  Insurance Office   0.2
1       Gas Station   0.2
2     Grocery Store   0.2
3              Bank   0.2
4  Department Store   0.2


----Clay County----
              venues  freq
0        Pizza Place  0.07
1  Convenience Store  0.07
2      Grocery Store  0.07
3        Coffee Shop  0.07
4               Pool  0.03


----Clayton County----
                    venues  freq
0                  Airport   0.5
1            Moving Target   0.5
2                      ATM   0.0
3  New American Restaurant   0.0
4     Other Great Outdoors   0.0


----Cobb County----
                       venues  freq
0                        Pool  0.25
1  Construction & Landscaping  0.12
2           Electronics Store  0.12
3                         Gym  0.12
4              S

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [81]:
import numpy as np
num_top_venues = 10

indicators = ['st','nd','rd']

columns = ['County']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns .append('{}th Most Common Venue'.format(ind+1))
        
    
    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['County'] = Venues_grouped['County']

for ind in np.arange(Venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,County,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Broward County,Rental Car Location,Clothing Store,Ice Cream Shop,Pharmacy,Sandwich Place,Grocery Store,Hotel,Donut Shop,Fast Food Restaurant,Mobile Phone Shop
1,Cass County,Bank,Grocery Store,Insurance Office,Gas Station,Department Store,Yoga Studio,Event Service,Dumpling Restaurant,Electronics Store,English Restaurant
2,Clay County,Pizza Place,Convenience Store,Grocery Store,Coffee Shop,Gourmet Shop,Discount Store,Bank,Dog Run,Sushi Restaurant,Electronics Store
3,Clayton County,Airport,Moving Target,Yoga Studio,Event Space,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service
4,Cobb County,Pool,Construction & Landscaping,Mexican Restaurant,Lake,Science Museum,Electronics Store,Gym,Yoga Studio,Ethiopian Restaurant,Donut Shop


In [80]:
df_cities.head()

,index,zip,type,decommissioned,primary_city,state,county,timezone,area_codes,country,latitude,longitude,irs_estimated_population_2015
0,12964,30132,STANDARD,0,Dallas,GA,Paulding County,America/New_York,"678,770",US,33.91,-84.83,34040
1,12987,30157,STANDARD,0,Dallas,GA,Paulding County,America/New_York,"678,770",US,33.88,-84.87,42710
2,13079,30301,PO BOX,0,Atlanta,GA,Fulton County,America/New_York,404,US,33.85,-84.39,1009
3,13080,30302,PO BOX,0,Atlanta,GA,Fulton County,America/New_York,404,US,33.75,-84.38,774
4,13081,30303,STANDARD,0,Atlanta,GA,Fulton County,America/New_York,"470,404,678",US,33.75,-84.39,1820


In [82]:
# set number of clusters
kclusters = 5

Venues_grouped_clustering = Venues_grouped.drop('County', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 3, 0, 1, 1, 1, 1, 1])

In [83]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [84]:
Venues_merged = df_cities

In [85]:
Venues_merged = Venues_merged.join(neighborhoods_venues_sorted.set_index('County'), on='county')

In [87]:
Venues_merged.columns

Index(['index', 'zip', 'type', 'decommissioned', 'primary_city', 'state',
       'county', 'timezone', 'area_codes', 'country', 'latitude', 'longitude',
       'irs_estimated_population_2015', 'Cluster Labels',
       '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [89]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Venues_merged['latitude'], Venues_merged['longitude'], Venues_merged['county'], Venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

In [92]:
Venues_merged.loc[Venues_merged['Cluster Labels'] == 0, Venues_merged.columns[[1] + list(range(5, Venues_merged.shape[1]))]]

,zip,state,county,timezone,area_codes,country,latitude,longitude,irs_estimated_population_2015,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,30339,GA,Cobb County,America/New_York,"404,470,678,770",US,33.86,-84.47,23860,0.0,Pool,Construction & Landscaping,Mexican Restaurant,Lake,Science Museum,Electronics Store,Gym,Yoga Studio,Ethiopian Restaurant,Donut Shop
196,64151,MO,Platte County,America/Chicago,816,US,39.21,-94.63,24670,0.0,Pool,Construction & Landscaping,Park,Gym,English Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
197,64152,MO,Platte County,America/Chicago,816,US,39.22,-94.72,26730,0.0,Pool,Construction & Landscaping,Park,Gym,English Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
198,64153,MO,Platte County,America/Chicago,816,US,39.27,-94.73,5210,0.0,Pool,Construction & Landscaping,Park,Gym,English Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
199,64154,MO,Platte County,America/Chicago,816,US,39.28,-94.64,9960,0.0,Pool,Construction & Landscaping,Park,Gym,English Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
206,64163,MO,Platte County,America/Chicago,816,US,39.34,-94.69,720,0.0,Pool,Construction & Landscaping,Park,Gym,English Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store
207,64164,MO,Platte County,America/Chicago,816,US,39.33,-94.63,250,0.0,Pool,Construction & Landscaping,Park,Gym,English Restaurant,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store


In [93]:
Venues_merged.loc[Venues_merged['Cluster Labels'] == 1, Venues_merged.columns[[1] + list(range(5, Venues_merged.shape[1]))]]

,zip,state,county,timezone,area_codes,country,latitude,longitude,irs_estimated_population_2015,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,30301,GA,Fulton County,America/New_York,404,US,33.85,-84.39,1009,1.0,Park,American Restaurant,Pool,Liquor Store,Gas Station,Coffee Shop,Light Rail Station,Rental Car Location,Burger Joint,Pizza Place
3,30302,GA,Fulton County,America/New_York,404,US,33.75,-84.38,774,1.0,Park,American Restaurant,Pool,Liquor Store,Gas Station,Coffee Shop,Light Rail Station,Rental Car Location,Burger Joint,Pizza Place
4,30303,GA,Fulton County,America/New_York,"470,404,678",US,33.75,-84.39,1820,1.0,Park,American Restaurant,Pool,Liquor Store,Gas Station,Coffee Shop,Light Rail Station,Rental Car Location,Burger Joint,Pizza Place
5,30304,GA,Fulton County,America/New_York,404,US,33.64,-84.39,0,1.0,Park,American Restaurant,Pool,Liquor Store,Gas Station,Coffee Shop,Light Rail Station,Rental Car Location,Burger Joint,Pizza Place
6,30305,GA,Fulton County,America/New_York,"678,404,470,770",US,33.83,-84.38,22030,1.0,Park,American Restaurant,Pool,Liquor Store,Gas Station,Coffee Shop,Light Rail Station,Rental Car Location,Burger Joint,Pizza Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,75395,TX,Dallas County,America/Chicago,214,US,32.79,-96.76,0,1.0,Hotel,Coffee Shop,Mexican Restaurant,Electronics Store,Piercing Parlor,Cocktail Bar,Sandwich Place,Convenience Store,Bar,Park
352,75396,TX,Dallas County,America/Chicago,214,US,32.78,-96.80,0,1.0,Hotel,Coffee Shop,Mexican Restaurant,Electronics Store,Piercing Parlor,Cocktail Bar,Sandwich Place,Convenience Store,Bar,Park
353,75397,TX,Dallas County,America/Chicago,214,US,32.79,-96.76,0,1.0,Hotel,Coffee Shop,Mexican Restaurant,Electronics Store,Piercing Parlor,Cocktail Bar,Sandwich Place,Convenience Store,Bar,Park
354,75398,TX,Dallas County,America/Chicago,"214,469",US,32.79,-96.76,0,1.0,Hotel,Coffee Shop,Mexican Restaurant,Electronics Store,Piercing Parlor,Cocktail Bar,Sandwich Place,Convenience Store,Bar,Park


In [94]:
Venues_merged.loc[Venues_merged['Cluster Labels'] == 2, Venues_merged.columns[[1] + list(range(5, Venues_merged.shape[1]))]]

,zip,state,county,timezone,area_codes,country,latitude,longitude,irs_estimated_population_2015,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,30132,GA,Paulding County,America/New_York,"678,770",US,33.91,-84.83,34040,2.0,Trail,Yoga Studio,Event Service,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space
1,30157,GA,Paulding County,America/New_York,"678,770",US,33.88,-84.87,42710,2.0,Trail,Yoga Studio,Event Service,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space


In [95]:
Venues_merged.loc[Venues_merged['Cluster Labels'] == 3, Venues_merged.columns[[1] + list(range(5, Venues_merged.shape[1]))]]

,zip,state,county,timezone,area_codes,country,latitude,longitude,irs_estimated_population_2015,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,30320,GA,Clayton County,America/New_York,404,US,33.63,-84.43,312,3.0,Airport,Moving Target,Yoga Studio,Event Space,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service


In [96]:
Venues_merged.loc[Venues_merged['Cluster Labels'] == 4, Venues_merged.columns[[1] + list(range(5, Venues_merged.shape[1]))]]

,zip,state,county,timezone,area_codes,country,latitude,longitude,irs_estimated_population_2015,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
288,75261,TX,Tarrant County,America/Chicago,214,US,32.79,-96.76,550,4.0,Piercing Parlor,Electronics Store,Yoga Studio,Event Space,Donut Shop,Dry Cleaner,Dumpling Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
